<a href="https://colab.research.google.com/github/hypercubex/experiment-pySpark/blob/main/Chapter_5_Loading_and_Saving_Data_in_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Saving Data in Spark

Collab Only code:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()

**Not on Colab you should start form HERE:**

Reading a text file textFile() in Python

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")

Loading all the .md files in one directory

In [ ]:
input = sc.textFile("spark-2.4.4-bin-hadoop2.7/*.md")

**Only in Google Colab:**

Load the example1.json JSON file (found on iCollege under Datasets)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
jsonDF = spark.read.json('example1.json')
jsonDF.printSchema()

# Saving Text Files

Using the USDA_activity_dataset_csv dataset (found on iCollege under Datasets)

**Only in Google Colab:**

Load the USDA file from Disk. 

NOTEL: Convert it to CSV on Excel first! 

Note: You might have to run this twice so it works fine.

In [ ]:
from google.colab import files
files.upload()

**Reading a CSV file into a DataFrame, filter some columns and save it**

In [ ]:
data = spark.read.csv('USDA_activity_dataset_csv.csv',inferSchema=True, header=True)

Filter data by several columns

In [ ]:
dataF=data.select("State","County","Median household income","Poverty rate")

Save only the filtered Data

In [ ]:
dataF.write.csv("USDA_income_poverty.csv")

Let's read this new file back into an RDD

In [ ]:
rddL=sc.textFile("USDA_income_poverty.csv")
rddL.collect()

# **Hive Example**

Using Hive to create and read a table - Simple Example

In [ ]:
from pyspark.sql import Row
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)
test_list = [('A', 25),('B', 20),('C', 25),('D', 18)]
rdd = sc.parallelize(test_list)
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
schemaPeople = sqlContext.createDataFrame(people)
# Register it as a temp table
sqlContext.registerDataFrameAsTable(schemaPeople, "test_table")
sqlContext.sql("show tables").show()

Let's query the table

In [ ]:
sqlContext.sql("Select * from test_table").show()

**Load a JSON file with Hive and use SQL on it**

In [ ]:
## Colab code only - DO NOT run outsie of colab
from google.colab import files  
files.upload()

Let's load example1.json with Hive a do a Select Statement on it

In [ ]:
from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)
ex1 = hiveCtx.read.json("example1.json")
ex1.registerTempTable("ex1")
results = hiveCtx.sql("SELECT ename, sal FROM ex1").show()